In [1]:
import io
import numpy as np

# Load Fasttext

In [2]:
# f = open('../crawl-300d-2M.vec/crawl-300d-2M.vec','r',encoding='utf8')
# content = f.readlines()
# f.close()

In [3]:
# def parser(content):
#     embedding = {}
#     ct = 0
#     failed_words = []
#     for c in content:
#         if ct % 5000 == 0:
#             print(ct)
#         c = c.rstrip()
#         line_content = c.split()
#         word = line_content[0]
#         rep_list = line_content[1:]
#         try:
#             rep = np.asarray(rep_list).astype('float')
#         except:
#             failed_words.append(ct)
#             ct+=1
#             continue
#         embedding[word] = rep
#         ct +=1
    
#     return embedding,failed_words

In [4]:
# fasttext_embedding, fasttext_failed_words = parser(content)

In [5]:
# import pickle
# f = open('fasttext_embedding.pckl','wb')
# pickle.dump(fasttext_embedding,f)
# f.close()

# Load GloVe

In [10]:
# import pickle

# f = open('../glove.840B.300d/glove.840B.300d.txt','r',encoding='utf8')

# content = f.readlines()

# f.close()

# glove_embedding = {}

# ct = 0
# failed_word = []
# for c in content:
#     if ct % 5000 == 0:
#         print(ct)
#     c = c.rstrip()
#     line_content = c.split()
#     word = line_content[0]
#     rep_list = line_content[1:]
#     try:
#         rep = np.asarray(rep_list).astype('float')
#     except:
#         failed_word.append(ct)
#         ct+=1
#         continue
#     glove_embedding[word] = rep
#     ct +=1 

# len(glove_embedding.keys())

# f = open('glove_embedding.pckl','wb')
# pickle.dump(glove_embedding,f)
# f.close()

In [2]:
import pickle
f = open('fasttext_embedding.pckl','rb')
fasttext_embedding = pickle.load(f)
f.close()

import pickle
f = open('glove_embedding.pckl','rb')
glove_embedding = pickle.load(f)
f.close()

In [4]:
vocab_glove = list(glove_embedding.keys())
vocab_fasttext = list(fasttext_embedding.keys())

In [5]:
vocab = list(set(vocab_fasttext) & set(vocab_glove))

In [6]:
indices_all = list(range(len(vocab)))
import random
random.shuffle(indices_all)
train_indices = indices_all[:int(0.8*len(indices_all))]
test_indices = indices_all[int(0.8*len(indices_all)):]

In [7]:

f = open('train_indices.pckl','wb')
pickle.dump(train_indices,f)
f.close()

In [8]:

f = open('test_indices.pckl','wb')
pickle.dump(test_indices,f)
f.close()

In [13]:
vocab = list(glove_embedding.keys())

In [15]:
BATCH_SIZE = 500

In [23]:
def get_train_batch(BATCH_SIZE):
    X_train = np.zeros((500,300))
    Y_train = np.zeros((500,300))
    
    random.shuffle(train_indices)
    batch_idx = train_indices[:BATCH_SIZE]
    ct = 0
    for i in batch_idx:
        word = vocab[i]
        try:
            fasttext = fasttext_embedding[word]
            glove = glove_embedding[word]
        except:
            not_found_in_both.append(word)
            continue
        X_train[ct] = fasttext
        Y_train[ct] = glove
        ct += 1
    
    tensor_y=torch.from_numpy(Y_train).long()
    tensor_x=torch.from_numpy(X_train).float()
    
    return tensor_x,tensor_y

In [25]:
import torch
from torch.autograd import Variable

ModuleNotFoundError: No module named 'torch'

In [ ]:
losses=[]

D_in, H, D_out = 300, 500, 400, 300

x = Variable(tensor_x).type(torch.FloatTensor)
y = Variable(tensor_y,requires_grad=False).type(torch.LongTensor)

## Model Specification ##
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )

# model.cuda()
loss_fn = torch.nn.MSELoss()

In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print("Iteration \t Loss")
for t in range(10000):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t%1000==0:
        print(t, loss.data.tolist())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()